<a href="https://colab.research.google.com/github/mengwangk/dl-projects/blob/master/04_06_auto_ml_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated ML - Tuning

In [0]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [0]:
if COLAB:
  !pip install -U imblearn
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects

In [0]:
if COLAB:
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .

In [0]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math 
import matplotlib
import sys

from scipy import stats
from collections import Counter
from pathlib import Path

plt.style.use('fivethirtyeight')

sns.set(style="ticks")

import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, precision_recall_curve, roc_curve, mean_squared_error, accuracy_score, average_precision_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
import pylab as pl
from collections import Counter

# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer

# from sklearn.ensemble import RandomForestClassifier

# from scikitplot.plotters import plot_precision_recall_curve

from dateutil.relativedelta import relativedelta

from IPython.display import display

from utils import *
from preprocess import *

import xgboost as xgb

np.set_printoptions(threshold=sys.maxsize)

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

from utils import feature_selection, plot_feature_importances
from plot import plot_correlation_matrix, plot_labelled_scatter

In [0]:
%aimport

## Preparation

In [0]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')

In [0]:
if COLAB:
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
else:
  DATASET_PATH = Path("datasets")
  ORIGIN_DATASET_PATH = Path('datasets')

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

if COLAB:
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M
  !ls -l dl-projects/datasets --block-size=M

In [0]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [0]:
data.info()

## Exploratory Data Analysis

### View data

In [0]:
# Feature matrix
feature_matrix = data.drop(columns=['NumberId', 'month', 'year'])

In [0]:
# Sort data
feature_matrix.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)

In [0]:
print('Positive: ' + str(feature_matrix['Label'].value_counts()[0]) + ' which is ', round(feature_matrix['Label'].value_counts()[0]/len(feature_matrix) * 100,2), '% of the dataset')
print('Negative: ' + str(feature_matrix['Label'].value_counts()[1]) + ' which is ', round(feature_matrix['Label'].value_counts()[1]/len(feature_matrix) * 100,2), '% of the dataset')

In [0]:
plt.figure(figsize=(8, 8))
sns.countplot('Label', data=feature_matrix)

In [0]:
feature_matrix.isna().sum().sort_values(ascending=False)

In [0]:
feature_matrix[feature_matrix.isnull().any(axis=1)].head()

### Data Cleansing

In [0]:
## Fill all NaN with 0
feature_matrix = feature_matrix.fillna(0)

In [0]:
feature_matrix.isna().sum().sort_values(ascending=False)

In [0]:
feature_matrix[feature_matrix.isnull().any(axis=1)].head()

### Feature Selection

In [0]:
# Feature scaling first??

In [0]:
print(feature_matrix.shape)
feature_matrix.columns

In [0]:
feature_matrix_selection = feature_selection(feature_matrix.drop(columns = ['time', 'TotalStrike', 'Label']))

In [0]:
feature_matrix_selection.shape, feature_matrix_selection.columns

In [0]:
feature_matrix_selection['time'] = feature_matrix['time']
feature_matrix_selection['TotalStrike'] = feature_matrix['TotalStrike']
feature_matrix_selection['Label'] = feature_matrix['Label']

### Feature Correlation

In [0]:
# Check without feature selection
# corrs = feature_matrix.corr().sort_values('Label')
# corrs['Label'].tail(100)

In [0]:
# Check with feature selection
corrs = feature_matrix_selection.corr().sort_values('Label')
corrs['Label'].tail(20)

### Balancing data

In [0]:
from imblearn.under_sampling import (RandomUnderSampler, 
                                     ClusterCentroids,
                                     TomekLinks,
                                     NeighbourhoodCleaningRule,
                                     AllKNN,
                                     NearMiss)
from imblearn.combine import SMOTETomek
from imblearn.pipeline import make_pipeline

pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'constant', fill_value=0)), ('scaler', StandardScaler())])

In [0]:
def under_sampling_test(feature_data, cut_off_date, ratio=0.8):
  y = feature_data.loc[feature_data['time'] < cut_off_date, 'Label']
  X = feature_data[feature_data['time'] < cut_off_date].drop(columns = ['Label', 'TotalStrike','time','date'], errors='ignore')
  total_count = y.value_counts()
  neg_count = y.value_counts()[0]
  pos_count = y.value_counts()[1]
  target_neg_count = int(pos_count / (1-ratio))
  target_ratio = {0: target_neg_count, 1:  pos_count}
  print(X.shape, y.shape, target_ratio)
  # print('Before sampling {}'.format(Counter(y)))
  sampler = RandomUnderSampler(sampling_strategy=target_ratio, random_state=42)
  X_bal, y_bal = sampler.fit_sample(X, y)
  print('Undersampling {}'.format(Counter(y_bal)))
  return X_bal, y_bal

# https://stackoverflow.com/questions/52499788/smotetomek-how-to-set-ratio-as-dictionary-for-fixed-balance
def balancing_pipeline_test(feature_data, cut_off_date, ratio=0.8):
  y = feature_data.loc[feature_data['time'] < cut_off_date, 'Label']
  X = feature_data[feature_data['time'] < cut_off_date].drop(columns = ['Label', 'TotalStrike','time','date'], errors='ignore')
  
  total_count = y.value_counts()
  neg_count = y.value_counts()[0]
  pos_count = y.value_counts()[1]
  target_neg_count = int(pos_count / (1-ratio))
  target_ratio = {0: target_neg_count, 1:  pos_count}

  print('Before sampling {}'.format(Counter(y)))
  #sampler = NearMiss(sampling_strategy={0: target_neg_count}, n_jobs=4)
  sampler = SMOTETomek(sampling_strategy='auto')
  X = pipeline.fit_transform(X)
  X_bal, y_bal = sampler.fit_sample(X, y)
  print('Over and undersampling {}'.format(Counter(y_bal)))
  return X_bal, y_bal

%time X_res, y_res = balancing_pipeline_test(feature_matrix_selection, pd.datetime(2019,6,1))

In [0]:
def plot_this(X_rs,y_rs,method):
  # Use principal component to condense the 10 features to 2 features
  pca = PCA(n_components=2).fit(X_rs)
  pca_2d = pca.transform(X_rs)
  # Assign colors
  for i in range(0, pca_2d.shape[0]):
    if y_rs[i] == 0:
      c1 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='r', marker='o')
    elif y_rs[i] == 1:
      c2 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='g', marker='*')  
  pl.legend([c1, c2], ['Class 1', 'Class 2'])
  pl.title(method)
  pl.axis([-4, 5, -4, 4])  # x axis (-4,5), y axis (-4,4)
  pl.show()

# Plot 
plot_this(X_res, y_res, 'SMOTETomek')

In [0]:
def under_sampling(X, y, ratio=0.9):
  total_count = y.value_counts()
  neg_count = y.value_counts()[0]
  pos_count = y.value_counts()[1]

  target_neg_count = int(pos_count / (1-ratio))
  target_ratio = {0: target_neg_count, 1:  pos_count}
  #print(X.shape, y.shape, target_ratio)
  print('Before sampling {}'.format(Counter(y)))
  sampler = RandomUnderSampler(random_state=42, sampling_strategy=target_ratio)
  X_bal, y_bal = sampler.fit_sample(X, y)
  print('Undersampling {}'.format(Counter(y_bal)))
  return X_bal, y_bal

def over_under_sampling(X, y):
  pass

## Modeling

In [0]:
def predict(dt, feature_matrix, sampling = False, return_probs = False): 
   
    feature_matrix['date'] = feature_matrix['time']

    # Subset labels
    test_labels = feature_matrix.loc[feature_matrix['date'] == dt, 'Label']
    train_labels = feature_matrix.loc[feature_matrix['date'] < dt, 'Label']

    print(f"Size of test labels {len(test_labels)}")
    print(f"Size of train labels {len(train_labels)}")
    
    # Features
    X_train = feature_matrix[feature_matrix['date'] < dt].drop(columns = ['NumberId', 'time',
                                                                                     'date', 'Label', 'TotalStrike', 'month', 'year', 'index'], errors='ignore')
    X_test = feature_matrix[feature_matrix['date'] == dt].drop(columns = ['NumberId', 'time',
                                                                                     'date', 'Label', 'TotalStrike', 'month', 'year', 'index'], errors='ignore')
    print(f"Size of X train {len(X_train)}")
    print(f"Size of X test  {len(X_test)}")
   
    feature_names = list(X_train.columns)
    
   
    # Impute and scale features
    pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'constant', fill_value=0)), ('scaler', StandardScaler())])

    # Fit and transform training data
    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(X_test)

    # Balance the data
    if sampling:
      X_train, train_labels = under_sampling(X_train, train_labels)
   
    # Labels
    y_train = np.array(train_labels).reshape((-1, ))
    y_test = np.array(test_labels).reshape((-1, ))
    
    print('Training on {} observations.'.format(len(X_train)))
    print('Testing on {} observations.\n'.format(len(X_test)))
    
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    # https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html
    # https://stats.stackexchange.com/questions/224512/reduce-false-positives-with-xgboost

    if type(train_labels) == np.ndarray:
      hit_ratio = float( len(np.where(train_labels == 0)[0]) / len(np.where(train_labels == 1)[0]) ) 
    else:
      hit_ratio = float(train_labels.value_counts()[0]/train_labels.value_counts()[1]) 
    print(f"Hit ratio - {hit_ratio}\n")

     # Create the classifier
    model = xgb.XGBClassifier(n_jobs=-1, 
                             random_state = 42,
                             n_estimators=100, 
                             max_depth=3,
                             objective='binary:logistic',
                             min_child_weight=1,
                             scale_pos_weight=hit_ratio 
                             )

    # Train 
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1]
    # Total positive
    positive = np.where((predictions==1))
    print(f'Total predicted to be positive: {len(positive[0])} \n')
  
    # Calculate metrics
    rpt = classification_report(y_test, predictions)
    cm = confusion_matrix(y_test, predictions)
    print('Classification report')
    print(rpt)
    print(f'Confusion matrix:\n {cm}\n')

    # Total predicted matches
    print('Predicted matches')
    pred = np.where((predictions==1))
    print(len(pred[0]), pred[0][0:23])
    topN = np.argpartition(probs, -23)[-23:]
    print(f'\n{topN}\n')  # Top N most high probability numbers
  
    if len(positive[0]) > 0:
    
      # Matching draws
      print('Matched draws')
      md = np.where((predictions==1) & (y_test==1))
      print(f"Count: {len(md[0])}, Index: {md}")
      month_data = feature_matrix.loc[feature_matrix['date'] == dt]
      numbers = month_data.iloc[md[0]][['MAX(Results.LuckyNo)']]

      print('\n\nTop 23 Possibility')
      print(origin_data[(origin_data['DrawDate'].dt.year == dt.year) & 
                          (origin_data['DrawDate'].dt.month == dt.month) &  
                          (origin_data['LuckyNo'].isin(topN))].head(23))  
      
      print('\n\nFirst 23 Numbers')
      print(origin_data[(origin_data['DrawDate'].dt.year == dt.year) & 
                          (origin_data['DrawDate'].dt.month == dt.month) &  
                          (origin_data['LuckyNo'].isin(pred[0][0:23]))].head(23))    
             

      print('\n\nAll matched')
      print(origin_data[(origin_data['DrawDate'].dt.year == dt.year) & 
                          (origin_data['DrawDate'].dt.month == dt.month) &  
                          (origin_data['LuckyNo'].isin(numbers['MAX(Results.LuckyNo)']))].head(100))    
                                                  
    else:
      print('No luck this month')                 

    # Feature importances
    fi = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
    
    if return_probs:
        return fi, probs
    
    return fi

In [0]:
%time june_2019 = predict(pd.datetime(2019,6,1), feature_matrix_selection)

In [0]:
#normalized_fi = plot_feature_importances(june_2019)

In [0]:
# Loop through from June to Dec
# start_mt = pd.datetime(2019,6,1)
# how_many_mt = 7
# for i in range(how_many_mt):
#   month_to_predict = start_mt + relativedelta(months=i)
#   print(f"\n{month_to_predict}\n-------------------\n")
#   %time predict(month_to_predict, feature_matrix_selection)